In [13]:
# give 2 portforlio, the old one and the optimized new one, 
# calculate the stepwise change of the portfolio
# and prioritize the changes by the improvement of the portfolio

cur_investment = [

        {
            "stock": "GILD",
            "quantity": 55,
        },
        {
            "stock": "LLY",
            "quantity": 6.68,
        },
        {
            "stock": "NVR",
            "quantity": 0.52,
        },
        {
            "stock": "MCK",
            "quantity": 6.56,
        },
        {
            "stock": "STAN.L",
            "quantity": 196.2
        },
        {
            "stock": "JKHY",
            "quantity": 30
        },
        {
            "stock": "601225.SS",
            "quantity": 800
        },
        {
            "stock": "600276.SS",
            "quantity": 300
        },
        {
            "stock": "MRK",
            "quantity": 11.67
        },
        {
            "stock": "VRTX",
            "quantity": 3.07
        },
        {
            "stock": "WMT",
            "quantity": 37.4
        },
        {
            "stock": "CTRA",
            "quantity": 64.99
        }
    ]


In [14]:
# compute the portfolio value

from util import get_currency_pair, read_and_filter_exchange_rates

tot_value = 0
for stock in cur_investment:
    # get the price of the stock which is the last line in the stock price file
    file_path = './data/prices/' + stock["stock"] + '.csv'
    with open(file_path) as f:
        lines = f.readlines()
        stock_price = float(lines[-1].split(",")[1])
        # if the price is zero, use the previous price
        if stock_price == 0:
            stock_price = float(lines[-2].split(",")[1])

    if '.' in stock["stock"]:
        stock_suffix = '.' + stock["stock"].split(".")[-1]
        # convert GBX to GBP
        if stock_suffix == '.L':
            stock_price *= 0.01
        exchange_name, needs_inversion, exchange_name_yahoo = get_currency_pair(stock_suffix, "USD")
        df_rate = read_and_filter_exchange_rates(exchange_name, exchange_name_yahoo)
        #print(df_rate)
        rate = df_rate[exchange_name].iloc[-1]

        if needs_inversion:
            rate = 1 / rate

        print(stock["stock"], stock_price, stock_price * rate, stock["quantity"] * stock_price * rate)
        tot_value += stock["quantity"] * stock_price * rate
        stock['value'] = stock["quantity"] * stock_price * rate
    else:
        print(stock["stock"], stock_price, stock["quantity"] * stock_price)
        tot_value += stock["quantity"] * stock_price
        stock['value'] = stock["quantity"] * stock_price


print(tot_value)


GILD 83.05000305175781 4567.75016784668
LLY 921.5 6155.62
NVR 9469.169921875 4923.968359375001
MCK 519.0800170898438 3405.1649121093747
HOLX 82.30999755859375 2079.1505383300782
STAN.L 7.607999877929688 10.053517938476215 1972.5002195290335
JKHY 176.8300018310547 5304.900054931641
601225.SS 22.170000076293945 3.1301180822997194 2504.0944658397752
600276.SS 44.95000076293945 6.3463603835486735 1903.908115064602
MRK 118.97000122070312 1388.3799142456055
VRTX 486.4200134277344 1493.3094412231444
WMT 79.76000213623047 2983.0240798950194
CTRA 22.979999542236328 1493.4701702499387
40175.24043863989


In [15]:
# compute the weight
for stock in cur_investment:
    stock['weight'] = stock['value'] / tot_value

cur_investment

[{'stock': 'GILD',
  'quantity': 55,
  'value': 4567.75016784668,
  'weight': 0.11369565229666907},
 {'stock': 'LLY',
  'quantity': 6.68,
  'value': 6155.62,
  'weight': 0.15321924480829305},
 {'stock': 'NVR',
  'quantity': 0.52,
  'value': 4923.968359375001,
  'weight': 0.12256226236891934},
 {'stock': 'MCK',
  'quantity': 6.56,
  'value': 3405.1649121093747,
  'weight': 0.084757797960416},
 {'stock': 'HOLX',
  'quantity': 25.26,
  'value': 2079.1505383300782,
  'weight': 0.05175203721569729},
 {'stock': 'STAN.L',
  'quantity': 196.2,
  'value': 1972.5002195290335,
  'weight': 0.04909740920011806},
 {'stock': 'JKHY',
  'quantity': 30,
  'value': 5304.900054931641,
  'weight': 0.13204401509516478},
 {'stock': '601225.SS',
  'quantity': 800,
  'value': 2504.0944658397752,
  'weight': 0.06232929631533401},
 {'stock': '600276.SS',
  'quantity': 300,
  'value': 1903.908115064602,
  'weight': 0.04739008638846762},
 {'stock': 'MRK',
  'quantity': 11.67,
  'value': 1388.3799142456055,
  'weig

In [16]:
import pandas as pd
import numpy as np

data_dir = "./processed_data_128"
S = pd.read_pickle(f'{data_dir}/S.pkl')
mu = np.load(f'{data_dir}/mu.npy')

# load valid_tickers.txt
with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
    valid_tickers = f.read().splitlines()


In [17]:
current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]

In [18]:
def portfolio_volatility(weights, covariance_log_returns):
    covariance_returns = np.exp(covariance_log_returns) - 1
    return np.sqrt(np.dot(weights.T, np.dot(covariance_returns, weights)))

def portfolio_return(weights, log_returns, allow_short=False):
    returns = np.exp(log_returns) - 1
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)


In [19]:
print(portfolio_volatility(current_weight, S))
print(portfolio_return(current_weight, mu))

0.039477972941871844
0.14112660610916367


In [20]:
from scipy.optimize import minimize
import json

def portfolio_volatility_log_return(weights, covariance):
    return np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

def portfolio_log_return(weights, returns, allow_short=False):
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)


def min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short=False):
    portfolio_ret = portfolio_log_return(weights, returns, allow_short)
    portfolio_vol = portfolio_volatility_log_return(weights, covariance)
    sharpe_ratio = (portfolio_ret - risk_free_rate) / portfolio_vol
    return -sharpe_ratio # Negate Sharpe ratio because we minimize the function


def optimize_portfolio(returns, covariance, risk_free_rate, allow_short=False):
    num_assets = len(returns)
    args = (returns, covariance, risk_free_rate)

    # Define constraints
    def constraint_sum(weights):
        return np.sum(weights) - 1
    
    constraints = [{'type': 'eq', 'fun': constraint_sum}]

    bounds = tuple((0.0, 0.20) for _ in range(num_assets))

    # Perform optimization
    def objective(weights):
        return min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short)
    
    iteration = [0]  # mutable container to store iteration count
    def callback(weights):
        iteration[0] += 1
        
        print(f"Iteration: {iteration[0]}, value: {objective(weights)}")

    # Initial guess (equal weights)
    initial_guess = num_assets * [1. / num_assets]

    # Perform optimization
    result = minimize(objective, initial_guess, 
                      method='SLSQP', bounds=bounds, constraints=constraints, callback=callback, options={'maxiter': 300})

    return result

INTEREST_RATE = 0.0497    # Current interest rate accessible for USD
ANNUAL_TRADING_DAYS = 252

def do_optimization(mu, S, final_tickers, period, allow_short):
  riskfree_log_return = np.log(1 + INTEREST_RATE) * period / ANNUAL_TRADING_DAYS
  raw_weights = optimize_portfolio(mu, S, riskfree_log_return, allow_short)
  weights = raw_weights.x
  
  tickers_to_buy = []
  print(f'Starting optimization: allow_short: {allow_short}')
  for index, ticker_name in enumerate(final_tickers):
    weight = weights[index]
    if weight > 1e-3:
      print(f'index: {index} {ticker_name}: weight {weight} exp profit: {mu[index]}, variance: {S[ticker_name][ticker_name]}')
      ticker_info = {'id': ticker_name, 'weight': weight}
      tickers_to_buy.append(ticker_info)

  print(f'expected return in {period} trading days: {portfolio_return(weights, mu)}')
  print(f'volatility of the return in {period} trading days: {portfolio_volatility(weights, S)}')
  print(f'optimization finished for allow_short={allow_short}')
  # print tickers_to_buy in JSON format

  tickers_to_buy_json = json.dumps(tickers_to_buy, indent=4)
  print(tickers_to_buy_json)
  return tickers_to_buy

In [21]:
tickers_to_buy = do_optimization(mu, S, valid_tickers, period=128, allow_short=False)

Iteration: 1, value: -1.4777107370679465
Iteration: 2, value: -1.6065851562677227
Iteration: 3, value: -0.6008091425711545
Iteration: 4, value: -0.5992051012831279
Iteration: 5, value: -1.6418769764807706
Iteration: 6, value: -0.847468053021559
Iteration: 7, value: -1.4039866109351626
Iteration: 8, value: -1.5891852600675322
Iteration: 9, value: -1.9058109054110677
Iteration: 10, value: -1.1213742959981174
Iteration: 11, value: -1.5873609197653546
Iteration: 12, value: -1.5453093035303962
Iteration: 13, value: -1.9068228665219618
Iteration: 14, value: -2.4483604736047178
Iteration: 15, value: -2.5705455671405484
Iteration: 16, value: -2.8606245619357913
Iteration: 17, value: -2.479341337328605
Iteration: 18, value: -2.837699551725213
Iteration: 19, value: -2.5938277621805397
Iteration: 20, value: -2.956421898676583
Iteration: 21, value: -2.983733609665094
Iteration: 22, value: -2.8546399636193187
Iteration: 23, value: -3.023722547701789
Iteration: 24, value: -2.990878094742276
Iteratio

In [22]:
# to find the steps to optimize my current portfolio to the optimized portfolio
# we need to find the difference between the optimized portfolio and the current portfolio
# and then prioritize the changes by the improvement of the portfolio

old_portfolio = pd.DataFrame(cur_investment)
optimized_portfolio = pd.DataFrame(tickers_to_buy)

# Merge the two dataframes to compare weights
comparison = pd.merge(old_portfolio, optimized_portfolio, left_on='stock', right_on='id', how='outer')
comparison['stock'] = comparison['stock'].fillna(comparison['id'])
comparison.fillna(0, inplace=True)  # Assuming no stock means 0 weight
comparison['weight_difference'] = comparison['weight_y'] - comparison['weight_x']

#*comparison['new_quantity'] = comparison['quantity'] * comparison['weight_y'] / comparison['weight_x'] if comparison['weight_x'] != 0 else 0
print(comparison[['stock','weight_x', 'weight_y', 'weight_difference', 'quantity']])

        stock  weight_x  weight_y  weight_difference  quantity
0        GILD  0.113696  0.157569           0.043873     55.00
1         LLY  0.153219  0.075681          -0.077538      6.68
2         NVR  0.122562  0.047666          -0.074896      0.52
3         MCK  0.084758  0.019270          -0.065487      6.56
4        HOLX  0.051752  0.000000          -0.051752     25.26
5      STAN.L  0.049097  0.018415          -0.030683    196.20
6        JKHY  0.132044  0.200000           0.067956     30.00
7   601225.SS  0.062329  0.047125          -0.015204    800.00
8   600276.SS  0.047390  0.022475          -0.024915    300.00
9         MRK  0.034558  0.000000          -0.034558     11.67
10       VRTX  0.037170  0.030247          -0.006922      3.07
11        WMT  0.074250  0.062062          -0.012188     37.40
12       CTRA  0.037174  0.033559          -0.003615     64.99
13       BA.L  0.000000  0.028565           0.028565      0.00
14     FRAS.L  0.000000  0.024153           0.024153   

In [23]:
comparison

,stock,quantity,value,weight_x,id,weight_y,weight_difference
0,GILD,55.00,4567.750168,0.113696,GILD,0.157569,0.043873
1,LLY,6.68,6155.620000,0.153219,LLY,0.075681,-0.077538
2,NVR,0.52,4923.968359,0.122562,NVR,0.047666,-0.074896
3,MCK,6.56,3405.164912,0.084758,MCK,0.019270,-0.065487
4,HOLX,25.26,2079.150538,0.051752,0,0.000000,-0.051752
5,STAN.L,196.20,1972.500220,0.049097,STAN.L,0.018415,-0.030683
6,JKHY,30.00,5304.900055,0.132044,JKHY,0.200000,0.067956
7,601225.SS,800.00,2504.094466,0.062329,601225.SS,0.047125,-0.015204
8,600276.SS,300.00,1903.908115,0.047390,600276.SS,0.022475,-0.024915
9,MRK,11.67,1388.379914,0.034558,0,0.000000,-0.034558


In [24]:
# Initialize the weights array

current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]


original_sharpe = portfolio_log_return(current_weight, mu) / portfolio_volatility_log_return(current_weight, S)
print(f'Original Sharpe ratio: {original_sharpe}')
# Optimize one stock at a time and adjust other weights proportionally

new_weights = current_weight.copy()
for i in range(15):
    results = []
    print(f'Starting iteration {i} optimization...')
    for index, row in comparison.iterrows():
        new_weights_try = new_weights.copy()
        stock_name = comparison.loc[index, 'stock']
        stock_idx = valid_tickers.index(stock_name)
        # Adjust the targeted stock weight
        new_weights_try[stock_idx] = comparison.loc[index, 'weight_y']
        # Adjust the rest to ensure total sum remains 1
        total_adjusted_weights = np.sum(new_weights_try) - new_weights_try[stock_idx]
        scale_factor = (1 - new_weights_try[stock_idx]) / total_adjusted_weights
        for j in range(len(new_weights_try)):
            if j != stock_idx:
                new_weights_try[j] *= scale_factor

        new_sharpe = portfolio_log_return(new_weights_try, mu) / portfolio_volatility_log_return(new_weights_try, S)
        results.append((comparison.loc[index, 'stock'], new_sharpe, new_sharpe - original_sharpe, new_weights_try, comparison.loc[index, 'weight_difference']))

    results.sort(key=lambda x: x[2], reverse=True)


    stock_name = results[0][0]
    stock_idx = valid_tickers.index(stock_name)
    # Adjust the targeted stock weight
    new_weights = results[0][3]
    new_sharpe = results[0][1]
    # remove the row from comparison
    comparison = comparison[comparison['stock'] != stock_name]

    original_sharpe = new_sharpe

    print(f'Iteration {i} optimization result:')
    print(f'Optimized ticker: {stock_name}')
    print(f'Sharpe ratio: {results[0][1]}')
    print(f'Improvement: {results[0][2]}')
    print(f'Weight difference: {results[0][4]}')
    


Original Sharpe ratio: 3.423301633712084
Starting iteration 0 optimization...
Iteration 0 optimization result:
Optimized ticker: HOLX
Sharpe ratio: 3.4662696623990352
Improvement: 0.042968028686951154
Weight difference: -0.05175203721569729
Starting iteration 1 optimization...
Iteration 1 optimization result:
Optimized ticker: 600900.SS
Sharpe ratio: 3.49182251775458
Improvement: 0.025552855355544857
Weight difference: 0.08077754560353284
Starting iteration 2 optimization...
Iteration 2 optimization result:
Optimized ticker: GILD
Sharpe ratio: 3.5263220187029374
Improvement: 0.0344995009483573
Weight difference: 0.04387319983481626
Starting iteration 3 optimization...
Iteration 3 optimization result:
Optimized ticker: BA.L
Sharpe ratio: 3.5360640914878894
Improvement: 0.009742072784951983
Weight difference: 0.028564569260577932
Starting iteration 4 optimization...
Iteration 4 optimization result:
Optimized ticker: 601225.SS
Sharpe ratio: 3.5429002977360757
Improvement: 0.00683620624818

In [14]:
results-

[('TTWO',
  3.2152527363675176,
  0.007677229590891761,
  array([0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.03124322,
         0.        , 0.        , 0.        , 0.

In [13]:
optimized_portfolio

,id,weight
0,BA.L,0.023168
1,FRAS.L,0.037291
2,STAN.L,0.012661
3,AMGN,0.004777
4,ACGL,0.012257
5,CTRA,0.032321
6,LLY,0.066531
7,FICO,0.011387
8,GILD,0.163008
9,JBL,0.010057
